In [56]:
import os
import collections
import pandas as pd
import numpy as np
import happybase

In [12]:
df = pd.read_csv(os.path.join(r"/home/zhaodachuan/code/hx_recall_system/predict_user_attribute","age","data","res_df.csv"))
for flag in ["trade","position"]:
    df = df.merge(pd.read_csv(os.path.join(r"/home/zhaodachuan/code/hx_recall_system/predict_user_attribute",flag,"data","res_df.csv")))
df

,uid,age,trade,position
0,1,36~45,金融,企业一般管理人员
1,6,36~45,IT/移动互联网,其他（自由职业）
2,8,26~29,金融,企业普通员工
3,38,30~35,金融,高校学生
4,44,36~45,IT/移动互联网,企业普通员工
...,...,...,...,...
261362,2902765,30~35,IT/移动互联网,企业普通员工
261363,2902766,30~35,零售消费,其他（自由职业）
261364,2902770,30~35,IT/移动互联网,企业普通员工
261365,2902775,36~45,房地产,其他（自由职业）


In [15]:
#11808
income_base_dict = {
    '金融': 72496,
    'IT/移动互联网': 99972,
    '制造业': 74151,
    '房地产': 62068,
    '零售消费': 68441,
    '汽车': 53213,
    '电商': 70137,
    'IOT': 89462,
    '游戏': 65337,
    '医疗': 83337
}
income_base_dict["其他"] = int(np.average([v for v in income_base_dict.values()]) * 0.8)
income_base_dict

{'金融': 72496,
 'IT/移动互联网': 99972,
 '制造业': 74151,
 '房地产': 62068,
 '零售消费': 68441,
 '汽车': 53213,
 '电商': 70137,
 'IOT': 89462,
 '游戏': 65337,
 '医疗': 83337,
 '其他': 59089}

In [19]:
times_age_dict = {
    '36~45': 4,
    '26~29': 2,
    '30~35': 3,
    '21~25': 1,
    '46~50': 4.5,
    '~20': 0,
    '50~': 5
}    

In [20]:
times_position_dict = {
    '企业一般管理人员': 3, 
    '其他（自由职业）': 1, 
    '企业普通员工': 1, 
    '高校学生': 0, 
    '企业创始人/CEO': 20, 
    '事业单位员工': 1, 
    '企业业务决策层': 8
}    

In [23]:
income_list = []
uid_list = []
for index in df.index:
    uid = df.loc[index,"uid"]
    age = df.loc[index,"age"]
    position = df.loc[index,"position"]
    trade = df.loc[index,"trade"]
    uid_list.append(uid)
    income_list.append(income_base_dict[trade] * times_age_dict[age] * times_position_dict[position])

In [24]:
def income_bucket(income):
    if income < 80000:
        return "~8万"
    elif income <300000:
        return "8~30万"
    elif income < 1000000:
        return "30~100万"
    else:
        return "100~万"
income_list = [income_bucket(v) for v in income_list]

In [26]:
res = collections.Counter(income_list)
res 

Counter({'30~100万': 20150, '8~30万': 201431, '~8万': 33131, '100~万': 6655})

In [27]:
v_sum = np.sum([v for v in res.values()])
{k:v/v_sum for k,v in res.items()}

{'30~100万': 0.07709465999915827,
 '8~30万': 0.7706826033891042,
 '~8万': 0.12676045560457136,
 '100~万': 0.02546228100716617}

In [ ]:
# 本科及以上
# 大专
# 高中
# 其他

In [28]:
df["income"] = income_list
df

,uid,age,trade,position,income
0,1,36~45,金融,企业一般管理人员,30~100万
1,6,36~45,IT/移动互联网,其他（自由职业）,30~100万
2,8,26~29,金融,企业普通员工,8~30万
3,38,30~35,金融,高校学生,~8万
4,44,36~45,IT/移动互联网,企业普通员工,30~100万
...,...,...,...,...,...
261362,2902765,30~35,IT/移动互联网,企业普通员工,8~30万
261363,2902766,30~35,零售消费,其他（自由职业）,8~30万
261364,2902770,30~35,IT/移动互联网,企业普通员工,8~30万
261365,2902775,36~45,房地产,其他（自由职业）,8~30万


In [46]:
education_base_dict = {
    '金融': 3,
    'IT/移动互联网': 3,
    '制造业': 1,
    '房地产': 0,
    '零售消费': 0,
    '汽车': 1,
    '电商': 1,
    'IOT': 3,
    '游戏': 3,
    '医疗': 3,
    "其他": 0
}

In [47]:
delta_age_dict = {
    '36~45': 2,
    '26~29': 1,
    '30~35': 1,
    '21~25': 1,
    '46~50': 2,
    '~20': 0,
    '50~': 2
}    

In [48]:
delta_position_dict = {
    '企业一般管理人员': 3, 
    '其他（自由职业）': 0, 
    '企业普通员工': 1, 
    '高校学生': 3, 
    '企业创始人/CEO': 3, 
    '事业单位员工': 3, 
    '企业业务决策层': 3
}    

In [49]:
education_list = []
uid_list = []
for index in df.index:
    uid = df.loc[index,"uid"]
    age = df.loc[index,"age"]
    position = df.loc[index,"position"]
    trade = df.loc[index,"trade"]
    uid_list.append(uid)
    education_list.append(education_base_dict[trade] + delta_age_dict[age] + delta_position_dict[position])

In [50]:
def education_bucket(education):
    if education == 0 :
        return "其他"
    elif education == 1:
        return "高中"
    elif education == 2:
        return "大专"
    else:
        return "本科及以上"
education_list = [education_bucket(v) for v in education_list]

In [51]:
res = collections.Counter(education_list)
res 

Counter({'本科及以上': 152931, '大专': 71960, '高中': 35962, '其他': 514})

In [52]:
v_sum = np.sum([v for v in res.values()])
{k:v/v_sum for k,v in res.items()}

{'本科及以上': 0.5851197741107332,
 '大专': 0.2753216741210635,
 '高中': 0.1375919683816243,
 '其他': 0.001966583386579025}

In [53]:
df["education"] = education_list
df

,uid,age,trade,position,income,education
0,1,36~45,金融,企业一般管理人员,30~100万,本科及以上
1,6,36~45,IT/移动互联网,其他（自由职业）,30~100万,本科及以上
2,8,26~29,金融,企业普通员工,8~30万,本科及以上
3,38,30~35,金融,高校学生,~8万,本科及以上
4,44,36~45,IT/移动互联网,企业普通员工,30~100万,本科及以上
...,...,...,...,...,...,...
261362,2902765,30~35,IT/移动互联网,企业普通员工,8~30万,本科及以上
261363,2902766,30~35,零售消费,其他（自由职业）,8~30万,高中
261364,2902770,30~35,IT/移动互联网,企业普通员工,8~30万,本科及以上
261365,2902775,36~45,房地产,其他（自由职业）,8~30万,大专


In [60]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 261367 entries, 0 to 261366
Data columns (total 6 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   uid        261367 non-null  int64 
 1   age        261367 non-null  object
 2   trade      261367 non-null  object
 3   position   261367 non-null  object
 4   income     261367 non-null  object
 5   education  261367 non-null  object
dtypes: int64(1), object(5)
memory usage: 24.0+ MB


In [62]:
pool = happybase.ConnectionPool(size=100, host="47.110.125.69", port=9090, timeout=10000,protocol='binary')

In [63]:
income_list = []
uid_list = []
with pool.connection() as connection:
    table = connection.table("feature_center:user_profile")
    with table.batch() as b:
        for index in df.index:
            uid = df.loc[index,"uid"]
            age = df.loc[index,"age"]
            position = df.loc[index,"position"]
            trade = df.loc[index,"trade"]
            education = df.loc[index,"education"]
            income = df.loc[index,"income"]
            b.put(
                str(uid).encode(), 
                {
                    b'attribute:predict_age':age.encode(),
                    b'attribute:predict_position':position.encode(), 
                    b'attribute:predict_trade':trade.encode(), 
                    b'attribute:predict_education':education.encode(), 
                    b'attribute:predict_income':income.encode(),
                }
            )
